In [1]:
# imports
import numpy as np
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader

from utils import *
from torchinfo import summary


device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [25]:
# hyper parameters
learning_rate = 0.1
BATCH_SIZE = 100
EPOCHS = 200
dataset_size = 50000
# max length on input sequence
Tx = 30
# length of output sequence
Ty = 10



In [26]:
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(dataset_size)

100%|██████████████████████████████████████████████████████████████████████████| 50000/50000 [00:01<00:00, 43673.05it/s]


In [27]:
dataset[:10]

[('may 24 1892', '1892-05-24'),
 ('june-22-1859', '1859-06-22'),
 ('april 12 1974', '1974-04-12'),
 ('19 march 1983', '1983-03-19'),
 ('wednesday august 5 1953', '1953-08-05'),
 ('sunday june 26 1881', '1881-06-26'),
 ('tuesday october 24 1843', '1843-10-24'),
 ('30 december 1827', '1827-12-30'),
 ('sunday july 25 2010', '2010-07-25'),
 ('tuesday may 19 1992', '1992-05-19')]

In [28]:
X_org, Y_org, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)
# Xoh and Yoh is one hot representation X and Y 

In [29]:
Xoh = torch.tensor(Xoh, dtype=torch.float32)
Y = torch.tensor(Y_org, dtype=torch.long)


/tmp/ipykernel_1655/1021108513.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y = torch.tensor(Y_org, dtype=torch.long)


In [30]:
class dataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.tensor(X, dtype=torch.float32).to(device)
        self.Y = torch.tensor(Y, dtype=torch.long).to(device)
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [31]:
train_data = dataset(Xoh, Y)

data_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

/tmp/ipykernel_1655/2152136669.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X, dtype=torch.float32).to(device)
/tmp/ipykernel_1655/2152136669.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.Y = torch.tensor(Y, dtype=torch.long).to(device)


In [9]:
n_a = 32         # hidden units for pre_attention_lstm/encoder
n_s = 64         # hidden units for post_attention_lstm/decoder

In [10]:
# repeat vector for attention mechanism
class RepeatVector(nn.Module):
    def __init__(self, n):
        super(RepeatVector, self).__init__()
        self.n = n

    def forward(self, x):
        # x: (batch_size, features)
        return x.unsqueeze(1).repeat(1, self.n, 1)

In [11]:
class modelf(nn.Module):
    def __init__(self, Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
        super(modelf, self).__init__()

        self.Tx = Tx
        self.Ty = Ty
        self.n_a = n_a
        self.n_s = n_s
        self.human_vocab_size = human_vocab_size
        self.machine_vocab_size = machine_vocab_size
        
        # one_step_attention layers:
        self.repeator = RepeatVector(self.Tx)
        repeator = RepeatVector(self.Tx)
        self.linear1 = nn.Linear((2*self.n_a) + self.n_s, 10)
        self.linear2 = nn.Linear(10, 1)
        
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

        # forward layers
        self.pre_attention_lstm = nn.LSTM(input_size=self.human_vocab_size, hidden_size=self.n_a, batch_first=True, bidirectional=True)
        self.post_attention_lstm = nn.LSTM(2* self.n_a , self.n_s, batch_first=True)
        self.output_layer = nn.Linear(self.n_s, self.machine_vocab_size)
        self.softmax_main = nn.Softmax(dim=-1)
    
    def one_step_attention(self, a, s_prev):
        # this attention mechnism for lstm
        
        # add's dimenison 1 making (1,2) to (1, 3, 2)
        s_prev = self.repeator(s_prev)
        concat = torch.cat([a, s_prev],dim=-1)
       
        e = self.linear1(concat)
        e = self.tanh(e)
        
        energies = self.linear2(e)
        energies = self.relu(energies)
        # softmax on dimension 1
        alphas = self.softmax(energies)
        # this is dot product
        context = torch.sum(alphas*a, dim=1, keepdim=True)
        
        return context


    def forward(self, X):
        
        outputs = []
        
        a, _ = self.pre_attention_lstm(X)

        batch_size = X.shape[0]
        s = torch.zeros(batch_size, n_s).to(device)
        c = torch.zeros(batch_size, n_s).to(device)

        for t in range(self.Ty):
            # one setp attention
            context = self.one_step_attention(a, s)
        
            _, (s, c) =  self.post_attention_lstm(context, (s.unsqueeze(0), c.unsqueeze(0)))
            s, c = s.squeeze(0), c.squeeze(0)
            
            out = self.output_layer(s)
            
            outputs.append(out)

        
        outputs = torch.stack(outputs, dim=1)
        return outputs

        
    

In [12]:
human_vocab_size = len(human_vocab)
machine_vocab_size = len(machine_vocab)
model = modelf(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size).to(device)

In [13]:
summary(model , input_size = Xoh.shape)

Layer (type:depth-idx)                   Output Shape              Param #
modelf                                   [31064, 10, 11]           --
├─LSTM: 1-1                              [31064, 30, 64]           18,432
├─RepeatVector: 1-2                      [31064, 30, 64]           --
├─Linear: 1-3                            [31064, 30, 10]           1,290
├─Tanh: 1-4                              [31064, 30, 10]           --
├─Linear: 1-5                            [31064, 30, 1]            11
├─ReLU: 1-6                              [31064, 30, 1]            --
├─Softmax: 1-7                           [31064, 30, 1]            --
├─LSTM: 1-8                              [31064, 1, 64]            33,280
├─Linear: 1-9                            [31064, 11]               715
├─RepeatVector: 1-10                     [31064, 30, 64]           --
├─Linear: 1-11                           [31064, 30, 10]           (recursive)
├─Tanh: 1-12                             [31064, 30, 10]        

In [32]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for inputs, labels in data_loader:
        optimizer.zero_grad()
        out = model(inputs)

        output_flat = out.reshape(-1, out.shape[-1])
        labels_flat = labels.reshape(-1)

        loss = criterion(output_flat, labels_flat)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    if epoch % 50 == 0 or epoch == EPOCHS-1:
        print(f"epoch:{epoch}, loss:{round(total_loss/1000, 4)}")

epoch:0, loss:0.5941
epoch:50, loss:0.2805
epoch:100, loss:0.367
epoch:150, loss:0.4914


In [ ]:
torch.save(model.state_dict(), "models/model_weights_1100.pth")

In [ ]:
def translate(model, sentence):
    model.eval()
    x_enc = string_to_int(sentence, Tx, human_vocab)
    x_enc = np.array(list(map(lambda x: np.eye(len(human_vocab))[x], x_enc)))
    X_tensor = torch.tensor(x_enc, dtype=torch.float32).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(X_tensor)
        if isinstance(outputs, list):
            outputs = torch.stack(outputs, dim=1)
        preds = torch.argmax(outputs, dim=-1).squeeze(0).cpu().numpy()
    return ''.join(int_to_string(preds, inv_machine_vocab))


In [ ]:
ans = translate(model, "19 3 2003")
ans

In [ ]:
examples = ['3 May 1979', '5 Apr 09', '20th Feb 2016', 'Wed 10 Jul 2007', '30 SEPT 2027', '1 jnu 2030']
for example in examples:
    print(f"Input: {example} -> Output: {translate(model, example)}")